De forma sencilla se puede ver que la regla compuesta de Simpson compuesta $S_c(f)$ se escribe como:

$$S_c(f) = \displaystyle \frac{h}{3(2n)} \left [ f(x_0) + f(x_{2n}) + 2 \sum_{i=1}^{n-1}f(x_{2i}) + 4 \sum_{i=1}^{n}f(x_{2i-1})\right ] $$
con $h=b-a$ y $n$ número de subintervalos.

Nota: Los nodos para el caso de Simpson se obtienen con la fórmula: $x_i = a +\frac{i}{2}\hat{h}, \forall i=0,\dots,2n, \hat{h}=\frac{h}{n}$.

Lo siguiente sirve de ayuda. Consideramos un caso de $n=6$ subintervalos, $2n+1=13$ nodos de $x_0$ a $x_{12}$. Los siguientes índices de los nodos se multiplican por 4 y son los índices que el subproceso 0 trabajará:

In [1]:
begin=1
end=6

In [2]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


Los siguientes índices de los nodos se multiplican por 2 y son los índices que el subproceso 0 trabajará:

In [3]:
for i in range(begin+1,end+1,2):
    print(i)

2
4
6


In [4]:
begin=7
end=12

Los siguientes índices se multiplican por 4 y son los índices que el subproceso 1 trabajará:

In [5]:
for i in range(begin,end+1,2):
    print(i)

7
9
11



Los siguientes índices se multiplican por 2 y son los índices que el subproceso 1 trabajará:

In [6]:
for i in range(begin+1,end+1,2):
    print(i)

8
10
12


In [7]:
import math
from multiprocessing import Pool 
from scipy.integrate import quad
import time
import multiprocessing as mp

**¿Qué restricciones se deben colocar a las variables $n, ns\text{_}p$ y $p$ para que el programa siguiente obtenga resultados correctos?**


+ $n$ deber ser par y debe ser divisible por p
+ $p$ debe ser múltiplo de n
+ $ns\text{_}p$ debe ser entero

por ejemplo si pruebas con $n=6, p=2, ns\text{_}p=6$ obtienes un error relativo de $10^{-7}$.

In [8]:
a=0
b=1
h=b-a
f=lambda x: math.exp(-x**2)

In [9]:
p=2 #número de subprocesos a lanzar

In [10]:
#n=10**4 #número de subintervalos a lanzar
n=6
h_hat=h/n

In [11]:
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p

In [12]:
nt

12

In [13]:
ns_p

6.0

In [14]:
ns_p=int(ns_p)


In [15]:
def Scf_parallel(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    begin = (mi_id*ns_p)+1
    end = begin-1+ns_p
    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    sum1=4*sum1
    sum2=0
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) 
    end_time=time.time()

**En el bloque del with ¿por qué tenemos que dividir por $\frac{}{6n}$ en lugar de por $\frac{}{3n}$ y por qué tenemos que restar $f(b)$?**

+ Se debe multiplicar por $6n$ porque conforme a la fórmula que se muestra al principio, se debe considerar el máximo índice de los nodos, el cuál se obtiene al multiplicar $2n$.

+ El segundo loop que considera los nodos pares, multiplica la función evaluada en cada nodo por 2, es decir, $2f(x_{2i})$. En este loop se está considerando el último nodo $x_{12}$, que corresponde al extremo derecho del intervalo en donde se desea evaluar la integral, entonces $x_{12}=b$. Por lo tanto $f(b)$ está siendo siendo multiplicado por 2, lo que implica que se suma dos veces; sin embargo la fórmula indica que la función evaluada en los nodos extremos solamente debe sumarse una vez. De ahí que $f(b)$ deba ser restada en el bloque del with.

In [16]:
aprox

0.7468245263791945

In [17]:
obj, err = quad(f, a, b)

In [18]:
obj

0.7468241328124271

In [19]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [20]:
err_relativo(aprox,obj)

5.26987211722925e-07

**¿Qué pasa si $n$ no es par? ¿qué tendríamos que modificar en la implemmentación si no es par?**

**tip: haz una prueba con $n=5$ subintervalos y por tanto $2n+1=11$ nodos. Entonces $\hat{h}=\frac{1}{5}$ y considera extremo izquierdo y derecho $a=0,b=1$. Luego genera los nodos con la fórmula $x_i=a+\frac{i}{2}\hat{h}$, $i=0,...,2n$ ($2n=10$) y considera por ejemplo $p=2$ subprocesos. Checa cuáles sumas se les tiene que multiplicar por 2 y por 4. Luego distribuye en partes iguales así como está la implementación anterior en los dos subprocesos. Revisa cómo quedan distribuidos los nodos en los dos subprocesos y cómo inician los for loops ¿se multiplican correctamente los sumandos por 2 y por 4 en cada subproceso con la definición de begin y end de cada loop?**

Conforme a las sugerencias arriba mencionadas, consideremos los siguientes parámetros:

In [21]:
a = 0
b = 1
n = 5
p = 2
h = b-a
h_hat = h/n
nt = 2*n #variable de apoyo para definir ns_p
ns_p = int(nt/p)

print('número de intervalos:',n)
print('número de subprocesos:',p)
print('tamaño de subintervalo(h_hat):',h_hat)
print('nodos por subproceso:',ns_p)

número de intervalos: 5
número de subprocesos: 2
tamaño de subintervalo(h_hat): 0.2
nodos por subproceso: 5


De acuerdo con estos parámetros, los índices con los que debería trabajar cada subproceso son los siguientes:

In [22]:
# Subproceso 0, nodos que se multiplican por 4

begin = 1
end = 5

print('Subproceso 0')
print('--Nodos que se multiplican por 4:')
for i in range(begin,end+1,2):
    print('  ',i)
print('--Nodos que se multiplican por 2:')
for i in range(begin+1,end,2):
    print('  ',i)

print('\nSubproceso 1')
print('--Nodos que se multiplican por 4:')
begin = 6
end = 10

for i in range(begin+1,end+1,2):
    print('  ',i)
print('--Nodos que se multiplican por 2:')
for i in range(begin,end+1,2):
    print('  ',i)

Subproceso 0
--Nodos que se multiplican por 4:
   1
   3
   5
--Nodos que se multiplican por 2:
   2
   4

Subproceso 1
--Nodos que se multiplican por 4:
   7
   9
--Nodos que se multiplican por 2:
   6
   8
   10


Revisemos ahora las siguientes líneas de código que corresponden a los índices de los nodos en la función Scf_parallel:

In [23]:
# Prueba de funcionamiento de loops en la función Scf_parallel para nodos pares e impares 

for i in range(p):
    begin = (i*ns_p) + 1
    end = begin -1 + ns_p
    print('\nSubproceso:',i)
    print('  desde subintervalo:',begin)
    print('  hasta:',end)
    print('     nodos multiplicados por 4')
    for k in range(begin, end+1, 2):
        print('-----nodo:',k)
    print('     nodos multiplicados por 2')
    for k in range(begin+1, end+1, 2):
        print('-----nodo:',k)


Subproceso: 0
  desde subintervalo: 1
  hasta: 5
     nodos multiplicados por 4
-----nodo: 1
-----nodo: 3
-----nodo: 5
     nodos multiplicados por 2
-----nodo: 2
-----nodo: 4

Subproceso: 1
  desde subintervalo: 6
  hasta: 10
     nodos multiplicados por 4
-----nodo: 6
-----nodo: 8
-----nodo: 10
     nodos multiplicados por 2
-----nodo: 7
-----nodo: 9


Como se puede observar, en el subproceso 0 los nodos impares son multiplicados por 4 y los pares son multiplicados por 2. En cambio, en el subproceso 1, los nodos pares son multiplicados por 4 y los impares por 2. Por lo tanto, no es correcta la forma en que se están asignando los índices en el subproceso 1.


Una forma de solucionar lo anterior podría ser redefiniendo el valor de begin (el cual indica el nodo a partir del cual cada subproceso empezará a trabajar) conforme a lo siguiente:

$begin = (mi_id * (ns\text{_}p -1)) + 1$

Utilizando esto en el ejemplo para n=5, el subproceso 0 trabajaría con los nodos del 1 al 5; y el subproceso 1 con los nodos del 5 al 9. En este caso ya no se consideran los nodos extremos 0 y 10, como se muestra en seguida:

In [24]:
# Prueba de modificación de loops para nodos pares e impares

for i in range(p):
    begin = (i*(ns_p-1))+1
    end = begin -1 + ns_p
    print('\nSubproceso:',i)
    print('  desde nodo:',begin)
    print('  hasta nodo:',end)
    print('     nodos multiplicados por 4:')
    for k in range(begin, end+1, 2):
        print('-----nodo:',k)
    print('     nodos multiplicados por 2:')
    for k in range(begin+1, end+1, 2):
        print('-----nodo:',k)


Subproceso: 0
  desde nodo: 1
  hasta nodo: 5
     nodos multiplicados por 4:
-----nodo: 1
-----nodo: 3
-----nodo: 5
     nodos multiplicados por 2:
-----nodo: 2
-----nodo: 4

Subproceso: 1
  desde nodo: 5
  hasta nodo: 9
     nodos multiplicados por 4:
-----nodo: 5
-----nodo: 7
-----nodo: 9
     nodos multiplicados por 2:
-----nodo: 6
-----nodo: 8


Notemos que en cada subproceso los nodos pares serán mutiplicados por 4 y los impares serán multiplicados por 2; sin embargo como el final del subproceso 0 es el nodo 5; y este mismo es el incio del suproceso 1, el nodo 5 es calculado en ambos procesos. Este nodo debe contribuir solamente una vez a la suma final. Por lo tanto, las modificaciones que deben realizarse a la función son las siguientes:

1. La variable begin debe modificarse conforme a $begin = (mi_id * (ns\text{_}p -1)) + 1 $
2. Definir el nodo 5, que de forma general corresponde a definir el nodo de en medio.
3. La evaluación del nodo 5, que es el nodo que queda justo en medio debe ser restada una vez al final del loop que incluye los nodos impares (los que son multiplicados por 4).
4. La variable **aprox** será calculada conforme a la fórmula original, es decir ya no deberá restarse $f(b)$, más bien deberá sumarse, porque los loops ya no incluyen al nodo final $f(b)$: $aprox = \frac{h}{6n}(f(a)+sum(results)+f(b))$

In [25]:
def Scf_parallel_impares(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    # 1. Modificación de begin
    begin = (mi_id*(ns_p-1))+1
    end = begin-1+ns_p
    # 2. Definición del nodo de en medio
    x_medio = a+n/2*h_hat
    sum1=0
    
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    # 3. Evaluación de f en x_medio y resta de la suma final
    f_medio = f(x_medio)
    sum1=4*(sum1-f_medio)
    sum2=0
    
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel_impares,range(p))
        #4. Modificación de la variable aprox conforme a la fórmula original
        aprox=h/(6*n)*(f(a)+sum(results)+f(b)) 
    end_time=time.time()

In [26]:
aprox

0.6429848438449228

In [27]:
err_relativo(aprox,obj)

0.1390411536066211

Se observa que para 5 subintervalos, el error relativo es de 0.13. Al incrementar el número de intervalos, este error disminuye, como se muestra en seguida:

In [28]:
n = 10005
p = 2
h = b-a
h_hat = h/n
nt = 2*n #variable de apoyo para definir ns_p
ns_p = int(nt/p)

print('número de intervalos:',n)
print('número de subprocesos:',p)
print('tamaño de subintervalo(h_hat):',h_hat)
print('nodos por subproceso:',ns_p)

número de intervalos: 10005
número de subprocesos: 2
tamaño de subintervalo(h_hat): 9.995002498750625e-05
nodos por subproceso: 10005


In [29]:
def Scf_parallel_impares(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    # 1. Modificación de begin
    begin = (mi_id*(ns_p-1))+1
    end = begin-1+ns_p
    # 2. Definición del nodo de en medio
    x_medio = a+n/2*h_hat
    sum1=0
    
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    # 3. Evaluación de f en x_medio y resta de la suma final
    f_medio = f(x_medio)
    sum1=4*(sum1-f_medio)
    sum2=0
    
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel_impares,range(p))
        #4. Modificación de la variable aprox conforme a la fórmula original
        aprox=h/(6*n)*(f(a)+sum(results)+f(b)) 
    end_time=time.time()

In [30]:
aprox

0.7467722387072734

In [31]:
err_relativo(aprox,obj)

6.948637955537197e-05